In [15]:
import os
import math
import tiktoken
import re
import pandas as pd
import numpy as np
import torch
import torch.nn as nn

In [25]:
source_file = '/Users/arjunkrishna/Desktop/tfr_llm/the-hundred-page-machine-learning--.txt'

with open(source_file, 'r', encoding='utf-8') as f:
    raw_data = f.read()

preprocessed = re.split(r'([,.:;?_!"()\']|--|\s)', raw_data)
preprocessed = [item.strip() for item in preprocessed if item.strip()]

# split into 90/10
n = len(preprocessed)
train_data = preprocessed[:int(n*0.9)]
val_data = preprocessed[int(n*0.9):]

train_text = " ".join(train_data) #'text': 'list' object cannot be converted to 'PyString' 
val_text = " ".join(val_data) # so we romove the list by using join

# tokenize using gpt2 
enc = tiktoken.get_encoding("gpt2")
train_ids = enc.encode_ordinary(train_text)
val_ids = enc.encode_ordinary(val_text)

torch.save(torch.tensor(train_ids), 'train.pt')
torch.save(torch.tensor(val_ids), 'train.pt')

print(f"train has {len(train_ids):,} tokens")
print(f"val has {len(val_ids):,} tokens")



train has 62,186 tokens
val has 6,902 tokens
